In [ ]:
# stdlib
import os

# syft absolute
import syft as sy
from syft.client.syncing import compare_states
from syft.client.syncing import resolve_single

In [ ]:
BIGQUERY_CREDENTIALS_FILE = ""
URL_HIGH = ""
PASSWORD_HIGH = ""

URL_LOW = ""
PASSWORD_LOW = ""

In [ ]:
# Local node + Client
# node_low = sy.orchestra.launch(
#     name="reddit_l",
#     node_side_type="low",
#     dev_mode=True,
#     reset=True,
#     local_db=True,
#     n_consumers=1,
#     create_producer=True,
# )

# node_high = sy.orchestra.launch(
#     name="reddit_h",
#     node_side_type="high",
#     dev_mode=True,
#     reset=True,
#     local_db=True,
#     n_consumers=1,
#     create_producer=True,
# )

# client_low = node_low.login(email="info@openmined.org", password="changethis")
# client_high = node_high.login(email="info@openmined.org", password="changethis")

In [ ]:
client_high = sy.login(
    url=URL_HIGH,
    email="info@openmined.org",
    password=PASSWORD_HIGH,
)

client_low = sy.login(
    url=URL_LOW,
    email="info@openmined.org",
    password=PASSWORD_LOW,
)

In [ ]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)

In [ ]:
client_low_ds = sy.login(
    url=URL_LOW,
    email="newuser@openmined.org",
    password="pw"
)

# Setup twin api

In [ ]:
import json

with open(BIGQUERY_CREDENTIALS_FILE, 'r') as f:
    bq_credentials = json.load(f)

In [ ]:
# Mock Behavior


@sy.mock_api_endpoint(
    # settings={}
    settings=bq_credentials,
)
def mock_query_function(
    context,  # Variable used to track user session, user information, user activities and settings
    sql_query: str,
) -> str:
    # third party
    from google.cloud import bigquery
    from google.oauth2 import service_account

    credentials = service_account.Credentials.from_service_account_info(
        context.settings
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )

    client = bigquery.Client(
        credentials=scoped_credentials,
        location="us-west1",
    )

    rows = client.query_and_wait(
        sql_query,
        project=context.settings["project_id"],
    )

    # Replacing private values to mocked ones.
    result = rows.to_dataframe()
    result["int64_field_0"] = 0
    result["id"] = "Private"
    result["name"] = "Private"
    result["subscribers_count"] = 0
    result["permalink"] = "Private"
    result["nsfw"] = "NaN"
    result["spam"] = False
    return result

In [ ]:
# Private Behavior


@sy.private_api_endpoint(
    # settings={}
    settings=bq_credentials,
)
def private_query_function(
    context,
    sql_query: str,
) -> str:
    # third party
    from google.cloud import bigquery
    from google.oauth2 import service_account

    print("test")
    credentials = service_account.Credentials.from_service_account_info(
        context.settings
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )

    client = bigquery.Client(
        credentials=scoped_credentials,
        location="us-west1",
    )
    print(client)

    rows = client.query_and_wait(
        sql_query,
        project=context.settings["project_id"],
    )
    print(rows)

    return rows.to_dataframe()

In [ ]:
new_endpoint = sy.TwinAPIEndpoint(
    path="reddit.query",
    private_function=private_query_function,
    mock_function=mock_query_function,
    description="Lorem ipsum dolor sit amet lorem adipiscing elit …",
)
client_high.api.services.api.add(endpoint=new_endpoint)

In [ ]:
client_high.refresh()

In [ ]:
client_high.api.services.reddit.query.private(
    sql_query="SELECT *  FROM test_1gb.subreddits LIMIT 100"
)

In [ ]:
twin_api_obj = client_high.api.services.api.api_endpoints()[0]

twin_api_obj

## Sync twin api

In [ ]:
#CustomEndpointActionObject

In [ ]:
high_state = client_high.get_sync_state()
low_state = client_low.get_sync_state()

high_state

In [ ]:
diff = compare_states(high_state, low_state)

diff

In [ ]:
widget = resolve_single(diff[0])

widget

In [ ]:
widget.click_sync()

In [ ]:
client_low.get_sync_state()

# Create request

## Use mock endpoint

In [ ]:
client_low.refresh()
client_low_ds.refresh()

In [ ]:
client_low_ds.api.services.reddit.query(
    sql_query="SELECT *  FROM test_1gb.subreddits LIMIT 100"
)

# Define code, project, request

In [ ]:
@sy.syft_function_single_use(
    reddit_query=client_low_ds.api.services.reddit.query,
)
def my_research_pipeline(reddit_query):
    sql_query = "SELECT *  FROM test_1gb.subreddits LIMIT 100"
    return reddit_query(sql_query=sql_query)

In [ ]:
new_project = sy.Project(
    name="Reddit Research Studies",
    description="Hi, I want to get information about your data.",
    members=[client_low_ds],
)

new_project.create_code_request(my_research_pipeline, client_low_ds)

In [ ]:
# low_request = client_low.requests[-1]

In [ ]:
# low_request

In [ ]:
# low_code = low_request.code

In [ ]:
# low_code

## Sync code request to high side

In [ ]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [ ]:
low_state

In [ ]:
diff_state = compare_states(low_state, high_state)

In [ ]:
diff_state

### Sync UserCode

In [ ]:
code_diff = diff_state[0]
widget = resolve_single(code_diff)

widget

In [ ]:
widget.click_sync()

### Sync Request

In [ ]:
request_diff = diff_state[1]
widget = resolve_single(request_diff)

widget

In [ ]:
widget.click_sync()

# High side: Run and sync back

## Run on high side

In [ ]:
client_high.refresh()

In [ ]:
client_high.requests

In [ ]:
request = client_high.requests[0]

In [ ]:
# request.code

In [ ]:
res = client_high.code.my_research_pipeline(
    reddit_query=client_high.api.services.reddit.query
)

In [ ]:
res

In [ ]:
request.accept_by_depositing_result(res)

## Sync back to low side

In [ ]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [ ]:
high_state

In [ ]:
diff_state_2 = compare_states(high_state, low_state)

In [ ]:
diff_state_2

In [ ]:
code_batch = diff_state_2[0]

widget = resolve_single(code_batch)
widget

In [ ]:
widget.click_sync()

In [ ]:
request_batch = diff_state_2[1]

widget = resolve_single(request_batch)
widget

In [ ]:
widget.click_sync()

In [ ]:
job_batch = diff_state_2[2]

widget = resolve_single(job_batch)
widget

In [ ]:
widget.click_share_all_private_data()

In [ ]:
widget.click_sync()

In [ ]:
client_low.get_sync_state()

# Run on low side

In [ ]:
client_low_ds.refresh()
client_low_ds.code.my_research_pipeline(
    reddit_query=client_low_ds.api.services.reddit.query
)